# Find additives which might enhance growth

In [9]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd

In [ ]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    new_medium.pop('EX_btn_e')
    new_medium['EX_nmn_e'] = 10.0 
    new_medium['EX_cobalt2_e'] = 10.0 
    new_medium['EX_pnto__R_e'] = 10.0
    new_medium['EX_arg__L_e'] = 10.0
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    pd.DataFrame(new_medium.items(), columns=['exchange', 'flux']).to_csv('../data/CGXII_comp_17.csv', index=False)
    sol = model.optimize()
sol

## strain 16

In [61]:
# loading
model = rg.load.load_model_cobra('../models/Cstr_16.xml')
CGXII_comp_16 = pd.read_csv('../data/CGXII_comp_16.csv').set_index('exchange').to_dict()['flux']

In [62]:
model.reactions.get_by_id('Growth')

Reaction identifier,Growth
Name,
Memory address,0x07fe8013768e0
Stoichiometry,0.000223 10fthf_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 54.124831 atp_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 0.0001... 0.000223 10-Formyltetrahydrofolate + 0.513689 L-Alanine + 0.000223 S-Adenosyl-L-methionine + 0.295792 L-Arginine + 0.241055 L-Asparagine + 0.241055 L-Aspartate + 54.124831 ATP C10H12N5O13P3 +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [63]:
with model:
    medium = model.medium
    model.medium = CGXII_comp_16
    sol = model.optimize()

base_growth = sol.objective_value
base_growth

0.5579056701595518

In [64]:
enhancement = {}
for ex in list(model.exchanges):
    with model:
        medium = model.medium
        CGXII_comp_16[ex.id] = 10.0
        model.medium = CGXII_comp_16
        sol = model.optimize()
        if sol.objective_value > base_growth:
            enhancement[ex.id] = sol.objective_value - base_growth
        CGXII_comp_16.pop(ex.id)

In [65]:
enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff'])
enh.sort_values(by=['diff'], ascending=False).head()#.to_csv('../analysis/additives/Cstr_16_adds.csv', index=False)

,exchange,diff
20,EX_arg__L_e,0.146284
13,EX_acgam_e,0.102785
1,EX_12dgr160_e,0.099626
2,EX_12dgr180_e,0.099626
3,EX_1ag160_e,0.099626


L-arginine and N-Acetyl-D-glucosamine are available in the lab storage

## strain 15

In [66]:
model = rg.load.load_model_cobra('../models/Cstr_15.xml')
CGXII_comp_15 = pd.read_csv('../data/CGXII_comp_15.csv').set_index('exchange').to_dict()['flux']

In [67]:
with model:
    medium = model.medium
    model.medium = CGXII_comp_15
    sol = model.optimize()
    
base_growth = sol.objective_value
base_growth

0.5522447060984086

In [68]:
enhancement = {}
for ex in list(model.exchanges):
    with model:
        medium = model.medium
        CGXII_comp_15[ex.id] = 10.0
        model.medium = CGXII_comp_15
        sol = model.optimize()
        if sol.objective_value > base_growth:
            enhancement[ex.id] = sol.objective_value - base_growth
        CGXII_comp_15.pop(ex.id)

In [69]:
enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff'])
enh.sort_values(by=['diff'], ascending=False).head()#.to_csv('../analysis/additives/Cstr_15_adds.csv', index=False)

,exchange,diff
0,EX_12dgr160_e,0.098615
2,EX_1ag160_e,0.098615
3,EX_1ag180_e,0.098615
4,EX_1ag181d9_e,0.098615
5,EX_1ag182d9d12_e,0.098615


## strain 14

In [70]:
model = rg.load.load_model_cobra('../models/Cstr_14.xml')
CGXII_comp_14 = pd.read_csv('../data/CGXII_comp_14.csv').set_index('exchange').to_dict()['flux']

In [71]:
with model:
    medium = model.medium
    model.medium = CGXII_comp_14
    sol = model.optimize()
    
base_growth = sol.objective_value
base_growth

0.5485482951479398

In [72]:
enhancement = {}
for ex in list(model.exchanges):
    with model:
        medium = model.medium
        CGXII_comp_14[ex.id] = 10.0
        model.medium = CGXII_comp_14
        sol = model.optimize()
        if sol.objective_value > base_growth:
            enhancement[ex.id] = sol.objective_value - base_growth
        CGXII_comp_14.pop(ex.id)

In [74]:
enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff'])
enh.sort_values(by=['diff'], ascending=False).to_csv('../analysis/additives/Cstr_14_adds.csv', index=False)